In [1]:
import os
import sys

import numpy as np
import torch
from torch import nn, autograd, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.models import efficientnet_v2_l
from torchvision.datasets import CIFAR10
import torchvision.transforms as T
from sklearn.metrics import accuracy_score
from tqdm import trange
import wandb
from torchsummary import summary

In [3]:
sys.path.append(os.path.dirname(os.getcwd()))
from pyfed.models.cnns import SimpleCNN
from pyfed.metrics.hessian import Hessian
from pyfed.metrics.basic import ClassificationMetrics

ImportError: cannot import name 'lr_lru_cache_wrapper' from 'functools' (/home/jahn/anaconda3/envs/torch_nightly/lib/python3.11/functools.py)

In [2]:
configs = {
    'seed':42,
    'epoch':100,
    'device':'cuda:0',
    'arch':efficientnet_v2_l,
}
mean, std = [0.49139968, 0.48215827 ,0.44653124], [0.24703233, 0.24348505, 0.26158768]
trainset = CIFAR10(
    '~/data', train=True,
    transform=T.Compose([
        T.RandomHorizontalFlip(),
        T.RandomRotation(90),
        T.Normalize(mean=mean, std=std),
        T.ToTensor()])
)
testset = CIFAR10(
    '~/data', train=False,
    transform=T.Compose([T.Normalize(mean=mean, std=std), T.ToTensor()])
)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=4)

In [4]:
device = torch.device(configs['device'])
metrics = ClassificationMetrics(n_classes=10, average=None)
set_random_state(configs['seed'])

net = configs['arch']()
init_state = net.state_dict()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
scaler = torch.cuda.amp.GradScaler()

project = 'hessian_distill'
group = 'train'
name = f'{net.__class__.__name__}_teacher'
tags = [net.__class__.__name__, trainset.__class__.__name__]
run = wandb.init(group=group, job_type='train', project=project, tags=tags, name=name, config=configs)

NameError: name 'ClassificationMetrics' is not defined

In [ ]:
for e in trange(configs['epoch']):
    net.train()
    metrics.reset()
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        with torch.cuda.amp.autocast():
            output = net(data)
            loss = criterion(output, target)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # Calc metrics
        targetcpu = target.cpu().numpy()
        predcpu = output.argmax(1).cpu().detach().numpy()
        metrics.update(targetcpu, predcpu)
    run.log(metrics.summary(prefix='Train'), commit=False)
    net.eval()
    metrics.reset()
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            with torch.cuda.amp.autocast():
                output = net(data)
            # Calc metrics
            targetcpu = target.cpu().numpy()
            predcpu = output.argmax(1).cpu().detach().numpy()
            metrics.update(targetcpu, predcpu)
    run.log(dict(metrics.summary(prefix='Test'), **{'epoch': e}))
run.finish()
torch.save(net.state_dict(), f'{name}.pth')